In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix



In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
df_meal

In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
#df_grouped_meal = df_grouped_meal.reset_index()
#df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'



In [ ]:
df_grouped_play.corr()

In [ ]:
len(speech_play.index)

In [ ]:
df_grouped_play

In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
# select only meal group from speech index
speech_meal = speech.xs('meal', level='group')
speech_play = speech.xs('play', level='group')

In [ ]:
speech_meal

In [ ]:
#df_grouped_meal = df_grouped_meal.reset_index(level=0, drop=True)
meal_data = pd.concat([df_grouped_meal, speech_meal], axis=1)
meal_data = meal_data.dropna()

In [ ]:
meal_data = meal_data.dropna()

In [ ]:
play_data = pd.concat([df_grouped_play, speech_play], axis=1)
#meal_data = meal_data.drop(columns=['index'])
play_data = play_data.dropna()

In [ ]:
play_data

# Classification with just movement

In [ ]:


# Perform a grid search for each classifier
X = df_grouped_meal[[
                     'cg_movement_mean', 'cg_movement_var',
       'cg_movement_min', 'cg_movement_max', 
       'child_movement_mean', 'child_movement_var',
       'child_movement_min', 'child_movement_max'
 ]]
y = df_grouped_meal['label']



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
len(play_data), len(df_grouped_play)

In [ ]:
play_data.corr()

## Speech child + movement play best performing model

In [ ]:

X = play_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max'
 ]]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = play_data['label'].iloc[:, 0]   


remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear', probability=True), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
conf_matrix_sum = np.zeros((2, 2))
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__pca__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }
    best_model_info = max(best_models.values(), key=lambda x: x['best_score'])
    best_model = best_model_info['best_estimator']

    # Use the best model to make predictions on the test set
    y_pred = best_model.predict(X_shuffled)

    # Compute the confusion matrix for this iteration
    conf_matrix = confusion_matrix(y_shuffled, y_pred)
    conf_matrix_sum += conf_matrix

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
average_conf_matrix = conf_matrix_sum / 10

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
import seaborn as sns
# Visualize the average confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(average_conf_matrix, annot=True, fmt=".2f", cmap="Blues", xticklabels=['no rad', 'rad'], yticklabels=['no rad', 'rad'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Average Confusion Matrix')
plt.show()

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=6)
cross_val_score(best_models['xgb']['best_estimator'] , X_meal, y_meal, cv=cv, scoring='accuracy').mean()

In [ ]:
pipeline_play = best_models['svc_linear']['best_estimator']


In [ ]:
pipeline_meal = best_models['xgb']['best_estimator']

In [ ]:
pipeline_meal

## check for significant difference between the models

In [ ]:
# the datasets:

X_meal = meal_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max'
 ]]
y_meal = meal_data['label'].iloc[:, 0]
X_play = play_data[['Proportion speech child', 'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

In [ ]:
acc_meal = []
acc_play = []
cv = KFold(n_splits=5, shuffle= False)
for i in range(10):
    # Shuffle the data with a different random state each time
    X_meal_shuffled, y_meal_shuffled = shuffle(X_meal, y_meal, random_state=i)
    X_play_shuffled, y_play_shuffled = shuffle(X_play, y_play, random_state=i)
    # Perform a grid search for each classifier
    acc_meal.append(cross_val_score(pipeline_meal, X_meal_shuffled, y_meal_shuffled, cv=cv, scoring='accuracy').mean())
    acc_play.append(cross_val_score(pipeline_play, X_play_shuffled, y_play_shuffled, cv=cv, scoring='accuracy').mean())
    

In [ ]:
acc_meal, acc_play

In [ ]:
from scipy.stats import ttest_rel, wilcoxon
t_stat, p_value = ttest_rel(acc_meal, acc_play)
t_stat, p_value

In [ ]:
X_meal = meal_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
         'average_proximity', 'variance_proximity', 'min_proximity', 'max_proximity'
 ]]
y_meal = meal_data['label'].iloc[:, 0]
X_play = play_data[['Proportion speech child', 'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

In [ ]:
from sklearn.model_selection import cross_val_predict
accs = []
f1s = []
precisions = []
recalls = []
for i in range(10):
    # Shuffle the data with a different random state each time
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    accs.append(cross_val_score(pipeline_play, X_meal, y_meal, cv=cv, scoring='accuracy').mean())
    y_pred = cross_val_predict(pipeline_play, X_meal, y_meal, cv=cv)
    f1s.append(f1_score(y_meal, y_pred))
    precisions.append(precision_score(y_meal, y_pred))
    recalls.append(recall_score(y_meal, y_pred))


In [ ]:
mean_acc = np.mean(accs)
std_acc = np.std(accs)

mean_f1 = np.mean(f1s)
std_f1 = np.std(f1s)

mean_precision = np.mean(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
std_recall = np.std(recalls)
print('Accuracy of model trained on play data evaluated on meal data')
print(f'Mean accuracy: {mean_acc}, Standard deviation: {std_acc}')
print(f'Mean F1 score: {mean_f1}, Standard deviation: {std_f1}')
print(f'Mean precision: {mean_precision}, Standard deviation: {std_precision}')
print(f'Mean recall: {mean_recall}, Standard deviation: {std_recall}')

In [ ]:
from sklearn.model_selection import cross_val_predict
accs = []
f1s = []
precisions = []
recalls = []
for i in range(10):
    # Shuffle the data with a different random state each time
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    accs.append(cross_val_score(pipeline_play, X_meal, y_meal, cv=cv, scoring='accuracy').mean())
    y_pred = cross_val_predict(pipeline_play, X_meal, y_meal, cv=cv)
    f1s.append(f1_score(y_meal, y_pred))
    precisions.append(precision_score(y_meal, y_pred))
    recalls.append(recall_score(y_meal, y_pred))


In [ ]:
mean_acc = np.mean(accs)
std_acc = np.std(accs)

mean_f1 = np.mean(f1s)
std_f1 = np.std(f1s)

mean_precision = np.mean(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
std_recall = np.std(recalls)
print('Accuracy of model trained on play data evaluated on meal data')
print(f'Mean accuracy: {mean_acc}, Standard deviation: {std_acc}')
print(f'Mean F1 score: {mean_f1}, Standard deviation: {std_f1}')
print(f'Mean precision: {mean_precision}, Standard deviation: {std_precision}')
print(f'Mean recall: {mean_recall}, Standard deviation: {std_recall}')

In [ ]:
X = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = play_data['label'].iloc[:, 0]   

In [ ]:
result = permutation_importance(pipeline_play, X, y, n_repeats=30, random_state=0, scoring='accuracy')
feature_importances_play = pd.DataFrame({
    'Feature': X.columns,
    'Importance': result['importances_mean']
})


In [ ]:
X = meal_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max',
         'average_proximity', 'variance_proximity', 'min_proximity', 'max_proximity'
 ]]
y = meal_data['label'].iloc[:, 0]


In [ ]:
result = permutation_importance(pipeline_meal, X, y, n_repeats=30, random_state=0, scoring='accuracy')
feature_importances_meal = pd.DataFrame({
    'Feature': X.columns,
    'Importance': result['importances_mean']
})


In [ ]:
feature_importances_play = feature_importances_play.sort_values(by='Importance', ascending=False)
feature_importances_meal = feature_importances_meal.sort_values(by='Importance', ascending=False)

# Define a custom color palette
features = list(set(feature_importances_play['Feature']).union(set(feature_importances_meal['Feature'])))
palette = sns.color_palette('Set1', len(features))
color_mapping = {feature: palette[i] for i, feature in enumerate(features)}

# Function to map colors to features
def get_colors(data, color_mapping):
    return data['Feature'].map(color_mapping).tolist()

# Plot the feature importances for Play
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances_play, palette=get_colors(feature_importances_play, color_mapping))
plt.title('Feature Importances Play')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

# Plot the feature importances for Meal
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances_meal, palette=get_colors(feature_importances_meal, color_mapping))
plt.title('Feature Importances Meal')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()



## calculate proportion of outcome


In [ ]:
total_importance = feature_importances_meal['Importance'].sum()

proportion_speech_child = feature_importances_meal.loc[feature_importances_meal['Feature'] == 'Proportion speech child', 'Importance'].sum() / total_importance

# Calculate the proportion for the rest of the features combined
proportion_rest = feature_importances_meal.loc[feature_importances_meal['Feature'] != 'Proportion speech child', 'Importance'].sum() / total_importance

print(f"Proportion driven by 'Proportion speech child': {proportion_speech_child:.2%}")
print(f"Proportion driven by the rest of the features combined: {proportion_rest:.2%}")

# Plot the proportions
labels = ['Child Speech', 'Movement Features']
sizes = [proportion_speech_child, proportion_rest]
colors = ['#ff9999','#66b3ff']
explode = (0.1, 0)  # explode the first slice

plt.figure(figsize=(8, 8))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Proportion of Total Feature Importance Meal')
plt.show()

In [ ]:
total_importance = feature_importances_play['Importance'].sum()
proportion_speech_child = feature_importances_play.loc[feature_importances_play['Feature'] == 'Proportion speech child', 'Importance'].sum() / total_importance

# Calculate the proportion for the rest of the features combined
proportion_rest = feature_importances_play.loc[feature_importances_play['Feature'] != 'Proportion speech child', 'Importance'].sum() / total_importance

# Plot the proportions
labels = ['Child Speech', 'Movement Features']
sizes = [proportion_speech_child, proportion_rest]
colors = ['#ff9999','#66b3ff']
explode = (0.1, 0)  # explode the first slice

plt.figure(figsize=(8, 8))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Proportion of Total Feature Importance Play')
plt.show()

# Contribution of speech vs movement

# Feature importances by class

In [ ]:
X = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = play_data['label'].iloc[:, 0]   

In [ ]:
X

In [ ]:
X_rad = X_play.loc[ X_play['label'] == 1]
X_rad = X_rad[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
X_non_rad = X_play.loc[ X_play['label'] == 0]
X_non_rad = X_non_rad[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y_rad = y[y == 1]
y_non_rad = y[y == 0]
results_rad = []
results_non_rad = []
for i in range(10):
    results_rad.append(permutation_importance(pipeline_play, X_rad, y_rad, n_repeats=30, random_state=i, scoring='accuracy'))
    results_non_rad.append(permutation_importance(pipeline_play, X_non_rad, y_non_rad, n_repeats=30, random_state=i, scoring='accuracy'))

In [ ]:
np.abs(np.mean([result['importances_mean'] for result in results_rad], axis=0))

In [ ]:
feature_importances_rad = pd.DataFrame({
    'Feature': X_rad.columns,
    'Importance': np.abs(np.mean([result['importances_mean'] for result in results_rad], axis=0)),
    'Std': np.mean([result['importances_std'] for result in results_rad], axis=0)
}).sort_values(by='Importance', ascending=False)

feature_importances_non_rad = pd.DataFrame({
    'Feature': X_non_rad.columns,
    'Importance':  np.abs(np.mean([result['importances_mean'] for result in results_non_rad], axis=0)),
    'Std': np.mean([result['importances_std'] for result in results_non_rad], axis=0)
}).sort_values(by='Importance', ascending=False)

features = list(set(feature_importances_rad['Feature']).union(set(feature_importances_non_rad['Feature'])))
palette = sns.color_palette('Set1', len(features))
color_mapping = {feature: palette[i] for i, feature in enumerate(features)}
feature_importances_rad['Importance_lower'] = np.clip(feature_importances_rad['Importance'] - feature_importances_rad['Std'], 0, None)
feature_importances_rad['Importance_upper'] = feature_importances_rad['Importance'] + feature_importances_rad['Std']

feature_importances_non_rad['Importance_lower'] = np.clip(feature_importances_non_rad['Importance'] - feature_importances_non_rad['Std'], 0, None)
feature_importances_non_rad['Importance_upper'] = feature_importances_non_rad['Importance'] + feature_importances_non_rad['Std']


# Plot the feature importances for RAD class
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances_rad, palette=color_mapping)
plt.errorbar(feature_importances_rad['Importance'], feature_importances_rad['Feature'],
             xerr=[feature_importances_rad['Importance'] - feature_importances_rad['Importance_lower'],
                   feature_importances_rad['Importance_upper'] - feature_importances_rad['Importance']],
             fmt='none', c='black', capsize=5)
plt.title('Feature Importances for RAD Class')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

# Plot the feature importances for non-RAD class
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances_non_rad, palette=color_mapping)
plt.errorbar(feature_importances_non_rad['Importance'], feature_importances_non_rad['Feature'],
             xerr=[feature_importances_non_rad['Importance'] - feature_importances_non_rad['Importance_lower'],
                   feature_importances_non_rad['Importance_upper'] - feature_importances_non_rad['Importance']],
             fmt='none', c='black', capsize=5)
plt.title('Feature Importances for Non-RAD Class')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
feature_importances_non_rad

## Plot ROC curve

In [ ]:
pipeline_play

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Assuming best_models is a dictionary containing the best models
best_estimator = pipeline_play

# Predict probabilities for the positive class
y_pred_proba = best_estimator.predict_proba(X)[:,1]

# Compute ROC curve and ROC area
fpr, tpr, thresholds = roc_curve(y, y_pred_proba)
roc_auc = roc_auc_score(y, y_pred_proba)

# Plot ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
#plt.xlim([-0.02, 1.0])  # Adjust x-axis limits to provide more space on the left
#plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate', labelpad=5)  # Adjust labelpad to move the y-axis label to the left
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Movement feature differences per class 

In [ ]:
features_play = df_grouped_play[['cg_movement_max', 'cg_movement_min', 'cg_movement_mean', 'cg_movement_var',
 'child_movement_mean', 'child_movement_max', 'child_movement_var', 'child_movement_min', 'label']]
features_meal = df_grouped_meal[['cg_movement_max', 'cg_movement_min', 'cg_movement_mean', 'cg_movement_var',
                                 'child_movement_mean', 'child_movement_max', 'child_movement_var', 'child_movement_min', 'label']]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Create a figure with subplots
features = ['child_movement_max', 'child_movement_min', 'child_movement_mean', 'child_movement_var']
label_column = 'label'
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(features):
    sns.boxplot(x=label_column, y=feature, data=df_grouped_play, ax=axes[i])
    axes[i].set_title(f'{feature} by {label_column}')
    axes[i].set_xlabel('Label')
    axes[i].set_ylabel(feature)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Create a figure with subplots
features = ['child_movement_max', 'child_movement_min', 'child_movement_mean', 'child_movement_var']
label_column = 'label'
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(features):
    sns.boxplot(x=label_column, y=feature, data=df_grouped_meal, ax=axes[i])
    axes[i].set_title(f'{feature} by {label_column}')
    axes[i].set_xlabel('Label')
    axes[i].set_ylabel(feature)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
final_df = pd.concat([df_grouped_meal, df_grouped_play], axis=0)
final_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Create a figure with subplots
features = ['child_movement_max', 'child_movement_min', 'child_movement_mean', 'child_movement_var']
label_column = 'label'
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(features):
    sns.boxplot(x=label_column, y=feature, data=final_df, ax=axes[i])
    axes[i].set_title(f'{feature} by {label_column}')
    axes[i].set_xlabel('Label')
    axes[i].set_ylabel(feature)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(x=final_df['label'], y=final_df['average_proximity'], data=final_df)

## Testing for differences between features in the two classes 

In [ ]:
# For the combined dataset
from scipy.stats import mannwhitneyu

# Example data
# Assuming final_df is your DataFrame and 'label' is the column with the labels
feature = 'cg_movement_max'
label_column = 'label'

# Separate the data into two groups based on the labels
group_0 = final_df[final_df[label_column] == 0][feature]
group_1 = final_df[final_df[label_column] == 1][feature]

# Perform the Mann-Whitney U test
stat, p_value = mannwhitneyu(group_0, group_1)

print(f'Mann-Whitney U test statistic: {stat}')
print(f'Mann-Whitney U test p-value: {p_value}')

# Interpretation
alpha = 0.05
if p_value < alpha:
    print('There is a significant difference between the two groups.')
else:
    print('There is no significant difference between the two groups.')

In [ ]:
# For the separate datasets
# For the combined dataset
from scipy.stats import mannwhitneyu

# Example data
# Assuming final_df is your DataFrame and 'label' is the column with the labels
feature = 'cg_movement_var'
label_column = 'label'

# Separate the data into two groups based on the labels
group_0 = df_grouped_play[df_grouped_play[label_column] == 0][feature]
group_1 = df_grouped_play[df_grouped_play[label_column] == 1][feature]

# Perform the Mann-Whitney U test
stat, p_value = mannwhitneyu(group_0, group_1)

print(f'Mann-Whitney U test statistic: {stat}')
print(f'Mann-Whitney U test p-value: {p_value}')

# Interpretation
alpha = 0.05
if p_value < alpha:
    print('There is a significant difference between the two groups.')
else:
    print('There is no significant difference between the two groups.')

In [ ]:
len(X_meal)

## take the best performing model outside of the pipeline and test it and then validate it on the other dataset

In [ ]:
pipeline_meal

In [ ]:
result = permutation_importance(pipeline_meal, X_meal, y_meal, n_repeats=30, random_state=0, scoring='accuracy')


In [ ]:
feature_importances_df = pd.DataFrame({
    'Feature': X_meal.columns,
    'Importance': result['importances_mean']
})

In [ ]:
feature_importances_meal = feature_importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
X_meal

In [ ]:
result = permutation_importance(pipeline_play, X_play, y_play, n_repeats=30, random_state=0, scoring='accuracy')


In [ ]:
meal_data_selected = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group','label']]
play_data_selected = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group', 'label']]

# Find the common indices
common_indices = meal_data_selected.index.intersection(play_data_selected.index)

# Filter the DataFrames to keep only the common indices
meal_data_filtered = meal_data_selected.loc[common_indices]
play_data_filtered = play_data_selected.loc[common_indices]

#meal_data_filtered = meal_data_filtered.set_index('group', append=True)
#play_data_filtered = play_data_filtered.set_index('group', append=True)

# Concatenate the filtered DataFrames
combined_data = pd.concat([meal_data_filtered, play_data_filtered],axis=0)

In [ ]:
combined_data

In [ ]:
## correlations between the two models 
# Get predictions from the LR model
meal_predictions =  pipeline_meal.predict_proba(meal_data_filtered.iloc[:,:-3])[:,1]

# Get predictions from the MLP model
play_predictions = pipeline_play.predict_proba(play_data_filtered.iloc[:,:-3])[:,1]

In [ ]:
accuracy_meal = pipeline_play.score(X_meal, y_meal)
print(f'Accuracy of model trained on play data evaluated on meal data: {accuracy_meal}')

In [ ]:
y_pred = pipeline_play.predict(X_meal)
f1 = f1_score(y_meal, y_pred, average='weighted')
precision = precision_score(y_meal, y_pred, average='weighted')
recall = recall_score(y_meal, y_pred, average='weighted')
print(f1, precision, recall)

In [ ]:
from sklearn.model_selection import cross_val_predict
accs = []
f1s = []
precisions = []
recalls = []
for i in range(10):
    # Shuffle the data with a different random state each time
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    pipeline_play.fit(X_play, y_play)
    accs.append(cross_val_score(pipeline_play, X_meal, y_meal, cv=cv, scoring='accuracy').mean())
    y_pred = cross_val_predict(pipeline_play, X_meal, y_meal, cv=cv)
    f1s.append(f1_score(y_meal, y_pred))
    precisions.append(precision_score(y_meal, y_pred))
    recalls.append(recall_score(y_meal, y_pred))


In [ ]:
mean_acc = np.mean(accs)
std_acc = np.std(accs)

mean_f1 = np.mean(f1s)
std_f1 = np.std(f1s)

mean_precision = np.mean(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
std_recall = np.std(recalls)
print('Accuracy of model trained on play data evaluated on meal data')
print(f'Mean accuracy: {mean_acc}, Standard deviation: {std_acc}')
print(f'Mean F1 score: {mean_f1}, Standard deviation: {std_f1}')
print(f'Mean precision: {mean_precision}, Standard deviation: {std_precision}')
print(f'Mean recall: {mean_recall}, Standard deviation: {std_recall}')

In [ ]:
accuracy_play = pipeline_meal.score(X_play, y_play)
print(f'Accuracy of model trained on meal data evaluated on play data: {accuracy_play}')

In [ ]:
y_pred = pipeline_meal.predict(X_play)
f1 = f1_score(y_play, y_pred, average='weighted')
precision = precision_score(y_play, y_pred, average='weighted')
recall = recall_score(y_play, y_pred, average='weighted')
print(f1, precision, recall)

In [ ]:
accs = []
f1s = []
precisions = []
recalls = []
for i in range(10):
    # Shuffle the data with a different random state each time
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    pipeline_meal.fit(X_meal, y_meal)
    accs.append(cross_val_score(pipeline_meal, X_play, y_play, cv=cv, scoring='accuracy').mean())
    y_pred = cross_val_predict(pipeline_meal, X_play, y_play, cv=cv)
    f1s.append(f1_score(y_play, y_pred))
    precisions.append(precision_score(y_play, y_pred))
    recalls.append(recall_score(y_play, y_pred))

In [ ]:
mean_acc = np.mean(accs)
std_acc = np.std(accs)

mean_f1 = np.mean(f1s)
std_f1 = np.std(f1s)

mean_precision = np.mean(precisions)
std_precision = np.std(precisions)

mean_recall = np.mean(recalls)
std_recall = np.std(recalls)
print('Accuracy of model trained on meal data evaluated on play data')
print(f'Mean accuracy: {mean_acc}, Standard deviation: {std_acc}')
print(f'Mean F1 score: {mean_f1}, Standard deviation: {std_f1}')
print(f'Mean precision: {mean_precision}, Standard deviation: {std_precision}')
print(f'Mean recall: {mean_recall}, Standard deviation: {std_recall}')

In [ ]:
combined_data = combined_data.dropna()
combined_labels = combined_data['label'].iloc[:,0]

# Combining the two models

In [ ]:
combined_groups

## soft voting

In [ ]:
y_shuffled

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier


# Define the base models for voting
base_models = [
    ('play', pipeline_play),
    ('meal', pipeline_meal)
]



accs = []
f1s = []
precisions = []
recalls = []
# Create the voting classifier
voting_ensemble = VotingClassifier(estimators=base_models, voting='soft', weights = [0.6, 0.4])
for i in range(10):
    # Shuffle the data with a different random state each time
    X_meal_shuffled, y_meal_shuffled = shuffle(X_meal, y_meal, random_state=i)
    X_play_shuffled, y_play_shuffled = shuffle(X_play, y_play, random_state=i)
    
    # Combine the shuffled data
    X_combined = pd.concat([X_meal_shuffled, X_play_shuffled])
    y_combined = pd.concat([y_meal_shuffled, y_play_shuffled])
    
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    
    # Evaluate the voting ensemble using cross-validation
    accs.append(cross_val_score(voting_ensemble, X_combined, y_combined, cv=cv, scoring='accuracy').mean())
    y_pred = cross_val_predict(voting_ensemble, X_combined, y_combined, cv=cv)
    f1s.append(f1_score(y_combined, y_pred, average='weighted'))
    precisions.append(precision_score(y_combined, y_pred, average='weighted'))
    recalls.append(recall_score(y_combined, y_pred, average='weighted'))

# Evaluate the voting ensemble using cross-validation


# Print the evaluation results
print(f"F1 Score: {f1s.mean()}")
print(f"Precision: {precisions.mean()}")
print(f"Recall: {recalls.mean()}")
print(f"Accuracy: {accs.mean()}") 

In [ ]:
print(np.mean(accs), np.std(accs), np.mean(f1s), np.std(f1s), np.mean(precisions), np.std(precisions), np.mean(recalls), np.std(recalls))

In [ ]:
X_meal = meal_data[['Proportion speech child',
    'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
    'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

X_play = play_data[['Proportion speech child',
    'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
    'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

y_meal = meal_data['label'].iloc[:,0]


y_play = play_data['label'].iloc[:,0]

In [ ]:
X_meal = X_meal.add_suffix('_meal')
X_play = X_play.add_suffix('_play')

# explore performance per class

In [ ]:
cross_val_predict(pipeline_play, X_play, y_play, cv=cv)

In [ ]:
threshold, len(y_play_high_confidence)

In [ ]:
# Initialize lists to store the results for Play model
play_accuracy_list = []
play_precision_list = []
play_recall_list = []
play_f1_list = []

# Initialize lists to store the results for Meal model
meal_accuracy_list = []
meal_precision_list = []
meal_recall_list = []
meal_f1_list = []

# Run the analysis 10 times with different fold splits
for i in range(10):
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    
    # Play model analysis
    play_preds = cross_val_predict(pipeline_play, play_data_filtered.iloc[:, :-3], combined_labels, cv=cv, method='predict_proba')
    play_confidences = np.max(play_preds, axis=1)
    threshold = np.percentile(play_confidences, 50)
    high_confidence_indices = play_confidences <= threshold
    play_preds_high_confidence = cross_val_predict(pipeline_play, play_data_filtered.iloc[:, :-3], combined_labels, cv=cv)[high_confidence_indices]
    y_play_high_confidence = combined_labels[high_confidence_indices]
    play_accuracy_list.append(accuracy_score(y_play_high_confidence, play_preds_high_confidence))
    play_precision_list.append(precision_score(y_play_high_confidence, play_preds_high_confidence, average=None))
    play_recall_list.append(recall_score(y_play_high_confidence, play_preds_high_confidence, average=None))
    play_f1_list.append(f1_score(y_play_high_confidence, play_preds_high_confidence, average=None))
    
    # Meal model analysis
    meal_preds = cross_val_predict(pipeline_meal, meal_data_filtered.iloc[:, :-3], combined_labels, cv=cv, method='predict_proba')
    meal_confidences = np.max(meal_preds, axis=1)
    threshold = np.percentile(meal_confidences, 50)
    high_confidence_indices = meal_confidences >= threshold
    meal_preds_high_confidence = cross_val_predict(pipeline_meal, meal_data_filtered.iloc[:, :-3], combined_labels, cv=cv)[high_confidence_indices]
    y_meal_high_confidence = combined_labels[high_confidence_indices]
    meal_accuracy_list.append(accuracy_score(y_meal_high_confidence, meal_preds_high_confidence))
    meal_precision_list.append(precision_score(y_meal_high_confidence, meal_preds_high_confidence, average=None))
    meal_recall_list.append(recall_score(y_meal_high_confidence, meal_preds_high_confidence, average=None))
    meal_f1_list.append(f1_score(y_meal_high_confidence, meal_preds_high_confidence, average=None))

# Convert lists to numpy arrays for easier averaging
play_precision_array = np.array(play_precision_list)
play_recall_array = np.array(play_recall_list)
play_f1_array = np.array(play_f1_list)
play_accuracy_array = np.array(play_accuracy_list)

meal_precision_array = np.array(meal_precision_list)
meal_recall_array = np.array(meal_recall_list)
meal_f1_array = np.array(meal_f1_list)
meal_accuracy_array = np.array(meal_accuracy_list)

# Calculate the average metrics across all iterations for Play model
avg_play_precision = play_precision_array.mean(axis=0)
avg_play_recall = play_recall_array.mean(axis=0)
avg_play_f1 = play_f1_array.mean(axis=0)
avg_play_accuracy = play_accuracy_array.mean()

# Calculate the average metrics across all iterations for Meal model
avg_meal_precision = meal_precision_array.mean(axis=0)
avg_meal_recall = meal_recall_array.mean(axis=0)
avg_meal_f1 = meal_f1_array.mean(axis=0)
avg_meal_accuracy = meal_accuracy_array.mean()

# Print the final results

print(f"Overall Play Accuracy: {avg_play_accuracy}")


print(f"Overall Meal Accuracy: {avg_meal_accuracy}")

## stacking

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
subset_pca = ['cg_movement_mean_meal',
       'cg_movement_var_meal', 'cg_movement_min_meal', 'cg_movement_max_meal',
       'child_movement_mean_meal', 'child_movement_var_meal',
       'child_movement_min_meal', 'child_movement_max_meal', 'cg_movement_mean_play',
       'cg_movement_var_play', 'cg_movement_min_play', 'cg_movement_max_play',
       'child_movement_mean_play', 'child_movement_var_play',
       'child_movement_min_play', 'child_movement_max_play']
remaining_features = [feat for feat in X_combined.columns if feat not in subset_pca]
results = []
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=4))
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])
# Define the meta-model
meta_models = [
    ('svc', Pipeline([('preprocessor', preprocessor), ('svc', SVC(probability=True))])),
    ('logistic_regression', Pipeline([('preprocessor', preprocessor), ('logistic_regression', LogisticRegression())])),
    ('random_forest', Pipeline([('preprocessor', preprocessor), ('random_forest', RandomForestClassifier())]))
]
param_grids = {
    'svc': {
        'svc__C': [0.01, 0.1, 1, 10, 100],
        'svc__gamma': [0.01, 0.1, 1, 10, 100],
        'svc__kernel': ['linear', 'rbf']
    },
    'logistic_regression': {
        'logistic_regression__C': [0.01, 0.1, 1, 10, 100],
        'logistic_regression__penalty': ['l1', 'l2'],
        'logistic_regression__solver': ['liblinear', 'saga']
    },
    'random_forest': {
        'random_forest__n_estimators': [10, 50, 100, 200],
        'random_forest__max_depth': [None, 5, 10, 15],
        'random_forest__min_samples_split': [2, 5, 10]
    }
}


for i in range(10):
    # Shuffle the data with a different random state each time
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    X_meal = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'average_proximity', 'variance_proximity', 'min_proximity', 'max_proximity',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

    X_play = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
    meal_predictions = cross_val_predict(pipeline_meal, X_meal, y_meal, cv=cv, method='predict_proba')
    play_predictions = cross_val_predict(pipeline_play, X_play, y_play, cv=cv, method='predict_proba')

    meal_predictions_df = pd.DataFrame(meal_predictions, columns=['meal_class_0', 'meal_class_1'], index=X_meal.index)

    play_predictions_df = pd.DataFrame(meal_predictions, columns=['play_class_0', 'play_class_1'], index=X_meal.index)

    combined_predictions = pd.concat([meal_predictions_df, play_predictions_df], axis=1).dropna()
    combined_labels = pd.concat([y_meal, y_play]).dropna()
    
    X_meal['group'] = 'meal'
    X_play['group'] = 'play'
    X_meal = X_meal.add_suffix('_meal')
    X_play = X_play.add_suffix('_play')
    X_combined = pd.concat([X_meal, X_play], axis=1)
    X_combined = pd.concat([X_combined, combined_predictions], axis=1)

    X_combined = X_combined.drop(columns=['group_meal', 'group_play'])
    X_combined = X_combined.dropna()

    combined_labels = pd.concat([y_meal, y_play],axis=1).dropna().iloc[:,0] 
    combined_groups = X_combined.index
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X_combined, combined_labels, combined_groups, random_state=i)
    cv = GroupKFold(n_splits=5)
    for model_name, meta_model in meta_models:
        grid_search = GridSearchCV(meta_model, param_grids[model_name], cv=cv, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)
        best_meta_model = grid_search.best_estimator_
        
        accuracy_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='accuracy', n_jobs=-1)
        precision_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='precision_weighted', n_jobs=-1)
        recall_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='recall_weighted', n_jobs=-1)
        f1_scores = cross_val_score(best_meta_model, X_shuffled, y_shuffled, groups=groups_shuffled, cv=cv, scoring='f1_weighted', n_jobs=-1)
        
        results.append({
            'random_state': i,
            'model': model_name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'f1_score': f1_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
results_df.groupby('model').mean()

# Looking at relationship between the two models

In [ ]:
## correlations between the two models 
# Get predictions from the LR model
meal_predictions =  pipeline_meal.predict_proba(meal_data_filtered.iloc[:,:-3])[:,1]

# Get predictions from the MLP model
play_predictions = pipeline_play.predict_proba(play_data_filtered.iloc[:,:-3])[:,1]



In [ ]:
# Compute the correlation between the predictions


In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
combined_labels = pd.concat([y_meal, y_play], axis=1).dropna().iloc[:,0]

In [ ]:
predictions_meal = cross_val_predict(pipeline_meal, meal_data_filtered.iloc[:,:-3], combined_labels, cv=cv)
predictions_play = cross_val_predict(pipeline_play, play_data_filtered.iloc[:,:-3], combined_labels, cv=cv)

## diversity of errors

In [ ]:
meal_accuracy = accuracy_score(combined_labels, predictions_meal)
play_accuracy = accuracy_score(combined_labels, predictions_meal)

# Measure the diversity of errors
meal_correct_play_incorrect = 0
play_correct_meal_incorrect = 0
both_incorrect = 0
for meal_pred, play_pred, true_label in zip(predictions_meal, predictions_play, combined_labels):
    if meal_pred == true_label and play_pred != true_label:
        meal_correct_play_incorrect += 1
    elif play_pred == true_label and meal_pred != true_label:
        play_correct_meal_incorrect += 1
    elif meal_pred != true_label and play_pred != true_label:
        both_incorrect += 1

# Calculate the proportions
total_samples = len(combined_labels)
meal_correct_play_incorrect_proportion = meal_correct_play_incorrect / total_samples
play_correct_meal_incorrect_proportion = play_correct_meal_incorrect / total_samples
both_incorrect_proportion = both_incorrect / total_samples

print(f"Proportion of samples where pipeline_meal is correct and pipeline_play is incorrect: {meal_correct_play_incorrect_proportion:.2%}")
print(f"Proportion of samples where pipeline_play is correct and pipeline_meal is incorrect: {play_correct_meal_incorrect_proportion:.2%}")
print(f"Proportion of samples where both models are incorrect: {both_incorrect_proportion:.2%}")

In [ ]:
meals = []
plays = []

for i in range(10):
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    meals.append(cross_val_predict(pipeline_meal, meal_data_filtered.iloc[:,:-3], combined_labels, cv=cv))
    plays.append(cross_val_predict(pipeline_play, play_data_filtered.iloc[:,:-3], combined_labels, cv=cv))

In [ ]:
from scipy.stats import pearsonr
correlations = []
for x, y in zip(meals, plays):
    correlation, p_value = pearsonr(x, y)
    correlations.append(correlation)

print(f"Correlation between meal and play model predictions: {np.mean(correlations)}")

In [ ]:
from sklearn.metrics import cohen_kappa_score
kappa_avg = []
# Calculate Cohen's Kappa
for x, y in zip (meals, plays):
    kappa_score = cohen_kappa_score(x, y)
    kappa_avg.append(kappa_score)
print("Cohen's Kappa score:", np.mean(kappa_avg))


In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import confusion_matrix
p_values = []
conf_matrix = []
for x, y in zip (meals, plays):
    contingency_table = confusion_matrix(x, y)
    result = mcnemar(contingency_table)
    p_values.append(result.pvalue)
    conf_matrix.append(contingency_table)


print("McNemar's test p-value:", np.mean(p_values))

In [ ]:
meals[0]

In [ ]:
cm = np.mean(conf_matrix, axis=0)
import seaborn as sns
# Visualize the average confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", xticklabels=['no rad', 'rad'], yticklabels=['no rad', 'rad'])
plt.xlabel('Meal')
plt.ylabel('Play')
plt.title('Confusion Matrix Meal vs Play')
plt.show()

In [ ]:
agreements = []
for x,y in zip(meals, plays):
    agreement_rate = np.mean(x == y)
    agreements.append(agreement_rate)

print("Prediction Agreement Rate:", np.mean(agreements))

# Getting the accuracy for the cases where the two classifiers output the same prediction

In [ ]:
meal_data_filtered.iloc[:,-1]

In [ ]:
predictions_play

In [ ]:
accuracies = []
precisions = []
recalls = []
f1_scores = []
lenghts = []
for x,y in zip(meals, plays):
    same_predictions_indices = x == y
    filtered_meal_data = meal_data_filtered[same_predictions_indices]
    filtered_play_data = play_data_filtered[same_predictions_indices]
    lenghts.append(len(filtered_meal_data))
    true_labels_meal = meal_data_filtered.iloc[:, -1][same_predictions_indices]
    true_labels_play = play_data_filtered.iloc[:, -1][same_predictions_indices]
    # Calculate accuracy score
    accuracy_meal = accuracy_score(true_labels_meal, x[same_predictions_indices])
    # Calculate precision, recall, and f1 score
    precision_meal = precision_score(true_labels_meal, x[same_predictions_indices], average='weighted')
    recall_meal = recall_score(true_labels_meal, x[same_predictions_indices], average='weighted')
    f1_meal = f1_score(true_labels_meal, x[same_predictions_indices], average='weighted')
    accuracies.append(accuracy_meal)
    precisions.append(precision_meal)
    recalls.append(recall_meal)
    f1_scores.append(f1_meal)


print(f"Accuracy for same predictions: {np.mean(accuracies)}")
print(f"Precision for same predictions: {np.mean(precisions)}")
print(f"Recall for same predictions: {np.mean(recalls)}")
print(f"F1 Score for same predictions: {np.mean(f1_scores)}")
print(f"Number of same predictions: {np.mean(lenghts)}")

In [ ]:
print(np.std(accuracies), np.std(precisions), np.std(recalls), np.std(f1_scores))

In [ ]:
30/49

# Train on disagreement zone

In [ ]:

X = play_data[['Proportion speech child','cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max'
 ]]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = play_data['label'].iloc[:, 0]   


remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear', probability=True), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
conf_matrix_sum = np.zeros((2, 2))
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': grid_search.best_params_.get('preprocessor__pca_pipeline__pca__n_components', None)
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

In [ ]:
X_meal.columns

In [ ]:
from sklearn.model_selection import cross_val_predict, KFold, GroupKFold, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier
import pandas as pd

results = []

classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear', probability=True), {
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf', probability=True), {
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]

for i in range(10):
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    meal_preds = cross_val_predict(pipeline_meal, meal_data_filtered.iloc[:, :-3], combined_labels, cv=cv)
    play_preds = cross_val_predict(pipeline_play, play_data_filtered.iloc[:, :-3], combined_labels, cv=cv)
    meal_probs = cross_val_predict(pipeline_meal, meal_data_filtered.iloc[:, :-3], combined_labels, cv=cv, method='predict_proba')
    play_probs = cross_val_predict(pipeline_play, play_data_filtered.iloc[:, :-3], combined_labels, cv=cv, method='predict_proba')
    dif_predictions_indices = meal_preds != play_preds

    # Create DataFrames for probabilities and predictions
    meal_results = pd.DataFrame(meal_probs, columns=['prob_0', 'prob_1'])
    meal_results['pred'] = meal_preds
    play_results = pd.DataFrame(play_probs, columns=['prob_0', 'prob_1'])
    play_results['pred'] = play_preds
    meal_results = meal_results[dif_predictions_indices]
    play_results = play_results[dif_predictions_indices]
    disagreements = pd.concat([meal_results, play_results], axis=0)
    true_labels_meal = meal_data_filtered.iloc[:, -1][dif_predictions_indices]
    true_labels_play = play_data_filtered.iloc[:, -1][dif_predictions_indices]
    disagreements_labels = pd.concat([true_labels_meal, true_labels_play], axis=0)
    X = disagreements
    y = disagreements_labels
    groups = disagreements.index
    cv = GroupKFold(n_splits=5)
    X_shuffled, y_shuffled, groups = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=cv, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups)

        # Calculate the cross-validated F1 score, precision, and recall
        # Store the results in a dictionary and add it to the list
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, groups=groups, cv=cv, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, groups=groups, cv=cv, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, groups=groups, cv=cv, scoring='recall_macro', n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, groups=groups, cv=cv, scoring='accuracy', n_jobs=-1)
        
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
disagreements['label'].value_counts()

In [ ]:
X

In [ ]:
weights = [0.6, 0.4]  # Adjust these weights based on model performance

# Initialize lists to store the evaluation metrics
f1_scores = []
precision_scores = []
recall_scores = []
accuracy_scores = []
gkf = GroupKFold(n_splits=5)
X = combined_data.iloc[:,:-3]
y = combined_data['label'].iloc[:,0]
groups = combined_data.index
# Perform manual cross-validation
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit the base models
    for name, model in base_models:
        model.fit(X_train, y_train)
    
    # Get the predicted probabilities from each base model
    meal_probs = pipeline_meal.predict_proba(X_test)[:, 1]
    play_probs = pipeline_play.predict_proba(X_test)[:, 1]
    
    # Combine the predicted probabilities using weighted averaging
    combined_probs = (weights[0] * meal_probs) + (weights[1] * play_probs)
    
    # Convert combined probabilities to class predictions
    combined_preds = np.where(combined_probs > 0.5, 1, 0)
    
    # Calculate evaluation metrics
    f1_scores.append(f1_score(y_test, combined_preds, average='macro'))
    precision_scores.append(precision_score(y_test, combined_preds, average='macro'))
    recall_scores.append(recall_score(y_test, combined_preds, average='macro'))
    accuracy_scores.append(accuracy_score(y_test, combined_preds))

# Print the evaluation results
print(f"F1 Score: {np.mean(f1_scores)}")
print(f"Precision: {np.mean(precision_scores)}")
print(f"Recall: {np.mean(recall_scores)}")
print(f"Accuracy: {np.mean(accuracy_scores)}")